In [1]:
from data import Dataset
from os.path import join
import pandas as pd
from configs import *
import re


df = Dataset(join(DATA, 'processed.xlsx'))

df.parse_datetime()
df.set_lag(lag=-1)

# df.set_lag(lag=1)
cols_for_delete = [
    
       # оставляем пока что П-_|Т нефти на входе|°С
       #  'П-1|Т нефти на входе|°С',
       #  'П-2|Т нефти на входе|°С',
       #  'П-3|Т нефти на входе|°С',
       # median аггрегация => П-123median|Т нефти на выходе|°С
       #  'П-1|Т нефти на выходе|°С',
       #  'П-2|Т нефти на выходе|°С',
       #  'П-3|Т нефти на выходе|°С',
       # max аггрегация => П-123max|Р нефти на входе|кгс/см²
       #  'П-1|Р нефти на входе|кгс/см²',
       #  'П-2|Р нефти на входе|кгс/см²',
       #  'П-3|Р нефти на входе|кгс/см²',
       # max аггрегация => П-123max|Р нефти на выходе|кгс/см²
       #  'П-1|Р нефти на выходе|кгс/см²',
       #  'П-2|Р нефти на выходе|кгс/см²',
       #  'П-3|Р нефти на выходе|кгс/см²',
       # константные значения
       #  'П-1|Р на горелку|кгс/см2',
       #  'П-2|Р на горелку|кгс/см3',
       #  'П-3|Р на горелку|кгс/см4',
       # БЕВ'ы 
       'БЕВ-1,L воды,см', 'БЕВ-1,V  воды,м3',
       'БЕВ-1,V нефти,м3', 'БЕВ-1,t воды,°С', 'БЕВ-2,L воды,см',
       'БЕВ-2,V  воды,м3', 'БЕВ-2,V нефти,м3', 'БЕВ-2,t воды,°С',
       'БЕВ-3,L воды,см', 'БЕВ-3,V  воды,м3', 'БЕВ-3,V нефти,м3',
       'БЕВ-3,t воды,°С',
    #    'Добыча воды за сутки|м3',
       ]

df.drop(cols_for_delete)
df.convert_datetime()

# df.new_feature('Р нефти на выходе', 'П-123max,Р нефти на выходе,кгс/см²', 'max')
# df.new_feature('Р нефти на входе', 'П-123max,Р нефти на входе,кгс/см²', 'max')
# df.new_feature('Т нефти на выходе', 'П-123median,Т нефти на выходе,кгс/см²', 'median')
# df.new_feature('Т теплоно-сителя', 'П-123median,Т теплоно-сителя,кгс/см²', 'max')
cols_for_smooth = df.cols[1:]
# df.save(join(DATA, 'without_recovery'))


In [2]:
# Графон
df.corr_matrix(cols_for_smooth, target=TARGET, title='origin',
               filepath=join(PLOTS, f'correlation/with_smoothed_all_methods.html'), append=True,
               figsize=(500, 2200))
###############################Smooth: rolling##################################
smooth_method='rolling'
df.smooth(cols_for_smooth, window=5, method=smooth_method) # insert=True
# матрица корреляций, без указания метода, высчитывает корреляцию спирмена, пирсона, кендела и фи-корреляцию
df.corr_matrix(smooth_method, target=TARGET, title=smooth_method,
               filepath=join(PLOTS, f'correlation/with_smoothed_all_methods.html'), append=True,
               figsize=(500, 2200))

for col in cols_for_smooth:
    df.difference_with_smoothed(col, smooth_method, figsize=(500, 1200), mode=None,  
                                filepath=join(PLOTS, 'smoothed_difference', smooth_method),
                                append=True
                                # show=True
                                )

##############################Smooth: lowess#####################################
smooth_method='lowess'
df.smooth(cols_for_smooth, frac=0.1, method=smooth_method) # insert=True
# матрица корреляций, без указания метода, высчитывает корреляцию спирмена, пирсона, кендела и фи-корреляцию
df.corr_matrix(smooth_method, target=TARGET, title=smooth_method,
               filepath=join(PLOTS, f'correlation/with_smoothed_all_methods.html'), append=True,
               figsize=(500, 2200))

# C-1, Рс сеп - 0.46
# 

for col in cols_for_smooth:
    df.difference_with_smoothed(col, smooth_method, figsize=(500, 1200), mode=None,  
                                filepath=join(PLOTS, 'smoothed_difference', smooth_method),
                                append=True
                                # show=True
                                )

##############################Smooth: savgol_filter#####################################
smooth_method='savgol_filter'
df.smooth(cols_for_smooth, window=10, polyorder=9, method=smooth_method) # insert=True
# матрица корреляций, без указания метода, высчитывает корреляцию спирмена, пирсона, кендела и фи-корреляцию
df.corr_matrix(smooth_method, target=TARGET, title=smooth_method,
               filepath=join(PLOTS, f'correlation/with_smoothed_all_methods.html'), append=True,
               figsize=(500, 2200))

# C-1, Рс сеп - 0.46
# 

for col in cols_for_smooth:
    df.difference_with_smoothed(col, smooth_method, figsize=(500, 1200), mode=None,  
                                filepath=join(PLOTS, 'smoothed_difference', smooth_method),
                                append=True
                                # show=True
                                )


interval columns not set, guessing: ['smoothed,Блок манифольда,P бм ,кгс/см²,rolling', 'smoothed,Блок манифольда,t жидкости,°С,rolling', 'smoothed,С-1,P сеп.,кгс/см²,rolling', 'smoothed,С-1,t жидкости,°С,rolling', 'smoothed,С-1,L жидкости,см,rolling', 'smoothed,С-2/1,P сеп. ,кгс/см²,rolling', 'smoothed,С-2/1,t жидкости,°С,rolling', 'smoothed,С-2/1,L межфазный,см,rolling', 'smoothed,С-2/1,L нефти,см,rolling', 'smoothed,С-2/2,P сеп. ,кгс/см²,rolling', 'smoothed,С-2/2,t жидкости,°С,rolling', 'smoothed,С-2/2,L межфазный,см,rolling', 'smoothed,С-2/2,L нефти,см,rolling', 'smoothed,ОН-1/1,P отс. ,кгс/см²,rolling', 'smoothed,ОН-1/1,t жидкости,°С,rolling', 'smoothed,ОН-1/1,L межфазный,см,rolling', 'smoothed,ОН-1/2,P отс. ,кгс/см²,rolling', 'smoothed,ОН-1/2,t жидкости,°С,rolling', 'smoothed,ОН-1/2,L  межфазный,см,rolling', 'smoothed,С-3,t жидкости,°С,rolling', 'smoothed,С-3,L нефти,см,rolling', 'smoothed,П-1,Т нефти на входе,°С,rolling', 'smoothed,П-1,Т нефти на выходе,°С,rolling', 'smoothed,П-1

In [5]:
df.time_series(cols_for_smooth, [TARGET], log=(False,True), filepath=join(PLOTS, 'time_series/origin.html'))
df.time_series('rolling', [TARGET], log=(False,True), filepath=join(PLOTS, 'time_series/rolling.html'))
df.time_series('lowess', [TARGET], log=(False,True), filepath=join(PLOTS, 'time_series/lowess.html'))
df.time_series('savgol_filter', [TARGET], log=(False,True), filepath=join(PLOTS, 'time_series/savgol_filter.html'))
